In [ ]:
""" import settings """
%load_ext autoreload
%autoreload 2

import numpy as np
import sys, os, csv

from utils_invitro_global import *


from utils_invitro_misc import parse_stim_filename, load_stim_csv
from utils_invitro_plot import stim_wavform_colors

In [ ]:
from matplotlib import pyplot as plt
from matplotlib_settings import set_plot_settings, reset_plot_settings

set_plot_settings()

In [ ]:
"""
Stimulation waveforms were measured using the following setup:
BISC stimulation current is injected into 1x PBS wells
Each well (MEA and Counter) have Ag/AgCl electrodes connected to BNC cable
BCN cable is connected to TIA (SR570) whose gain is configured to TIA_GAIN
TIA output is connected to an oscilloscope whose voltage measurement is read out and stored 
"""

In [ ]:
""" fetch all data files """
data_dir = f'{STIM_DATA_DIR}/amp_sweep'
file_list = [fn for fn in os.listdir(data_dir) if fn.endswith('csv')]

In [ ]:
""" how to parse filename """
# stimulation configurations have been directly embedded in the filename..
# this cell is an example of parsing a filename
# errata: although prog_set in the filename says 3
# sets 0, 1, 2, 3 were actually used, resulting in 4*(8x8) pixels.
# it means that all measurements were taken from a set of 256 (out of 16384) pixels

fn = 'ibias_global_6_ibias_stim_4_prog_set_4_polarity_0_negative_4_positive_4_balance_15_num_rep_1_num_spikes_3_extra_balance_10.csv'
ibias_global, ibias_stim, prog_set, polarity, negative, positive, balance, _, _, _ =  parse_stim_filename(fn)
print(ibias_global, ibias_stim, prog_set, polarity, negative, positive, balance)

Plot Amplitude Sweep Waveforms

In [ ]:
""" Expected Amplitudes """
# stimulation current amplitude is configured using
# two configuration params: ibias_global and ibias_stim
# amplitude follows formula: ibias_global*ibias_stim_map[ibias_stim] (uA)

y0_theory = []
ibias_stim_map = [0, 1, 2, 4, 6, 8, 12, 16]
ibias_stim = 7
for ibias_global in range(1, 8):
    y0_theory.append(10e-6*ibias_global*ibias_stim_map[ibias_stim])

# in theory, these are the currents we expect (uA)
print(np.array(y0_theory)/1e-6)

In [ ]:
""" pick IBIAS_STIM from 0, 1, ... , 7 """
IBIAS_STIM = 7

In [ ]:
""" Plot Cathodic First, for a fixed IBIAS_STIM """
plt.close('all')
fig, ax = plt.subplots(figsize=(4, 3))

T0 = 25e-6*(12.5/13.56)
color_idx = 0

for fn in file_list:
    ibias_global, ibias_stim, _, polarity, negative, positive, balance, _, _, _ = parse_stim_filename(fn)
    assert negative == positive # equal length bi-phasic
    pulse_length = negative

    # polarity = 0 indicates cathodic-first
    if not (polarity == 0 and ibias_stim == IBIAS_STIM):
        continue
    data = load_stim_csv(data_dir, fn)

    # measurement setup: BISC -> TIA -> Oscilloscope 
    # convert oscilloscope voltage to current
    t, v1 = np.array(data[:, 0]), np.array(data[:, 1])/TIA_GAIN

    # oscilloscope was triggered to a positive edge
    # apply time offset correction
    if polarity == 0:
        t = t - t[len(t)//2]
    else:
        t = t - t[len(t)//2] - (pulse_length*T0)

    # pick color. wrap around if needed
    color = stim_wavform_colors[color_idx%len(stim_wavform_colors)]
    color_idx += 1

    # plot
    ax.plot(t/1e-6, v1/1e-3, color=color)

ax.set_xlim((-125, 125))
ax.set_ylabel('Stim (mA)')
ax.set_xlabel('Time (μs)')
ax.grid(True)

In [ ]:
""" Plot Cathodic First, for a fixed IBIAS_STIM """
plt.close('all')
fig, ax = plt.subplots(figsize=(4, 3))

T0 = 25e-6*(12.5/13.56)
color_idx = 0

for fn in file_list:
    ibias_global, ibias_stim, _, polarity, negative, positive, balance, _, _, _ = parse_stim_filename(fn)
    assert negative == positive # equal length bi-phasic
    pulse_length = negative

    # polarity = 1 indicates anodic-first
    if not (polarity == 1 and ibias_stim == IBIAS_STIM):
        continue
    data = load_stim_csv(data_dir, fn)

    # measurement setup: BISC -> TIA -> Oscilloscope 
    # convert oscilloscope voltage to current
    t, v1 = np.array(data[:, 0]), np.array(data[:, 1])/TIA_GAIN

    # oscilloscope was triggered to a positive edge
    # apply time offset correction
    if polarity == 0:
        t = t - t[len(t)//2]
    else:
        t = t - t[len(t)//2] - (pulse_length*T0)

    # pick color. wrap around if needed
    color = stim_wavform_colors[color_idx%len(stim_wavform_colors)]
    color_idx += 1

    # plot
    ax.plot(t/1e-6, v1/1e-3, color=color)

ax.set_xlim((-125, 125))
ax.set_ylabel('Stim (mA)')
ax.set_xlabel('Time (μs)')
ax.grid(True)